In [1]:
import pandas as pd

from pathlib import Path
import csv

import os
import requests
import json

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
## IEX Constants
iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

## Redundant Assignment but improves Readability throughout code 
real_token = iex_api_key
test_token = iex_test_api_key

base_url_iex = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'

## IEX Status Test 
test_resp = requests.get(base_url_iex + 'status')
test_resp

<Response [200]>

In [4]:
token_status = test_token ## Set to either real token or test token for IEX

In [5]:
## Load IEX to get ETF statistics 
def get_IEX_statistics(stock_ticker, token=token_status):
    if token == test_token:
        resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/stats/?token='+test_token)
        data_json = resp_data.json()
    elif token == real_token:
        resp_data = requests.get(base_url_iex+'stock/'+stock_ticker+'/stats/?token='+real_token)
        data_json = resp_data.json()
        
    return data_json

In [6]:
## Use pickle module to import and export and save files
import pickle
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [11]:
## Import files using Pickle
imported_data_dict = {}
imported_key_list = []
## Need to set length manually - check files for max length based off file name 
def import_data(loop_length):
    x=0
    y=49
    increment=50
    
    while x < loop_length:
        str_symbol1 = str(x)
        str_symbol2 = str(y)

        pkl_path = Path('../FilesExportCompleteFMP_big/data_complete_'+str_symbol1+'_'+str_symbol2+'.pkl')
        data_obj = load_obj(pkl_path)
        data_obj_key_list = []
        for key in data_obj.keys():
            data_obj_key_list.append(key)  
        for symbol in data_obj_key_list:
            data = data_obj[symbol]
            imported_data_dict[symbol] = data

        x += increment
        y += increment
        if y > loop_length: y = loop_length
        
    for key in imported_data_dict.keys():
        imported_key_list.append(key) 

In [13]:
import_data(loop_length=199)

In [14]:
imported_data_dict

{'A':                   open        high         low       close    adjClose  \
 Date                                                                     
 2016-01-04   41.060001   41.189999   40.340000   40.689999   38.715168   
 2016-01-05   40.730000   40.950001   40.340000   40.549999   38.581966   
 2016-01-06   40.240002   40.990002   40.049999   40.730000   38.753220   
 2016-01-07   40.139999   40.150002   38.810001   39.000000   37.107185   
 2016-01-08   39.220001   39.709999   38.470001   38.590000   36.717079   
 ...                ...         ...         ...         ...         ...   
 2021-10-26  159.630000  159.880000  157.060100  158.190000  158.190000   
 2021-10-27  158.300000  158.320000  154.830000  155.260000  155.260000   
 2021-10-28  155.980000  157.790000  154.190000  155.760000  155.760000   
 2021-10-29  155.130000  157.570000  154.440000  157.490000  157.490000   
 2018-12-05    0.000000    0.000000    0.000000    0.000000    0.000000   
 
                vo